# Lesson 3: Chatbot Example

In this lesson, you will familiarize yourself with the chatbot example you will work on during this course. The example includes the tool definitions and execution, as well as the chatbot code. Make sure to interact with the chatbot at the end of this notebook.

## Import Libraries

In [1]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic

## Tool Functions

In [2]:
PAPER_DIR = "papers"

The first tool searches for relevant arXiv papers based on a topic and stores the papers' info in a JSON file (title, authors, summary, paper url and the publication date). The JSON files are organized by topics in the `papers` directory. The tool does not download the papers.  

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    """
    Search for papers on arXiv based on a topic and store their information.
    
    Args:
        topic: The topic to search for
        max_results: Maximum number of results to retrieve (default: 5)
        
    Returns:
        List of paper IDs found in the search
    """
    
    # Use arxiv to find the papers 
    client = arxiv.Client()

    # Search for the most relevant articles matching the queried topic
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # Create directory for this topic
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # Try to load existing papers info
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # Process each paper and add to papers_info  
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [4]:
search_papers("CNN satellite data")

Results are saved in: papers/cnn_satellite_data/papers_info.json


['2012.03093v1',
 '1704.06410v2',
 '2011.05586v2',
 '2112.01542v2',
 '1904.01994v1']

The second tool looks for information about a specific paper across all topic directories inside the `papers` directory.

In [6]:
def extract_info(paper_id: str) -> str:
    """
    Search for information about a specific paper across all topic directories.
    
    Args:
        paper_id: The ID of the paper to look for
        
    Returns:
        JSON string with paper information if found, error message if not found
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [7]:
extract_info('2012.03093v1')

'{\n  "title": "Semantic Segmentation of Medium-Resolution Satellite Imagery using Conditional Generative Adversarial Networks",\n  "authors": [\n    "Aditya Kulkarni",\n    "Tharun Mohandoss",\n    "Daniel Northrup",\n    "Ernest Mwebaze",\n    "Hamed Alemohammad"\n  ],\n  "summary": "Semantic segmentation of satellite imagery is a common approach to identify\\npatterns and detect changes around the planet. Most of the state-of-the-art\\nsemantic segmentation models are trained in a fully supervised way using\\nConvolutional Neural Network (CNN). The generalization property of CNN is poor\\nfor satellite imagery because the data can be very diverse in terms of\\nlandscape types, image resolutions, and scarcity of labels for different\\ngeographies and seasons. Hence, the performance of CNN doesn\'t translate well\\nto images from unseen regions or seasons. Inspired by Conditional Generative\\nAdversarial Networks (CGAN) based approach of image-to-image translation for\\nhigh-resolutio

## Tool Schema

Here are the schema of each tool which you will provide to the LLM.

In [35]:
# Set the client type: "anthropic" or "groq"
# Make sure to have the correct API key in your .env file
# ANTHROPIC_API_KEY for anthropic, GROQ_API_KEY for groq

CLIENT_TYPE = "groq"  # or "anthropic"


In [36]:
if CLIENT_TYPE == "anthropic":
    tools = [
        {
            "name": "search_papers",
            "description": "Search for papers on arXiv based on a topic and store their information.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "topic": {
                        "type": "string",
                        "description": "The topic to search for"
                    }, 
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of results to retrieve",
                        "default": 5
                    }
                },
                "required": ["topic"]
            }
        },
        {
            "name": "extract_info",
            "description": "Search for information about a specific paper across all topic directories.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "paper_id": {
                        "type": "string",
                        "description": "The ID of the paper to look for"
                    }
                },
                "required": ["paper_id"]
            }
        }
    ]
elif CLIENT_TYPE == "groq":
    tools = [
        {
            "type": "function",
            "function": {
                "name": "search_papers",
                "description": "Search for papers on arXiv based on a topic and store their information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "topic": {
                            "type": "string",
                            "description": "The topic to search for"
                        }, 
                        "max_results": {
                            "type": "integer",
                            "description": "Maximum number of results to retrieve",
                            "default": 5
                        }
                    },
                    "required": ["topic"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "extract_info",
                "description": "Search for information about a specific paper across all topic directories.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "paper_id": {
                            "type": "string",
                            "description": "The ID of the paper to look for"
                        }
                    },
                    "required": ["paper_id"],
                },
            },
        },
    ]
else:
    raise ValueError("Invalid CLIENT_TYPE. Choose 'anthropic' or 'groq'.")





## Tool Mapping

This code handles tool mapping and execution.

In [37]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "The operation completed but didn't return any results."
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## Chatbot Code

The chatbot handles the user's queries one by one, but it does not persist memory across the queries.

In [38]:
load_dotenv()

if CLIENT_TYPE == "anthropic":
    client = anthropic.Anthropic()
elif CLIENT_TYPE == "groq":
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
else:
    raise ValueError("Invalid CLIENT_TYPE. Choose 'anthropic' or 'groq'.")

### Query Processing

In [50]:
def process_query(query):
    
    messages = [{'role': 'user', 'content': query}] 

    if CLIENT_TYPE == "anthropic":
        response = client.messages.create(max_tokens = 2024,
                                  model = 'claude-3-7-sonnet-20250219', 
                                  tools = tools,
                                  messages = messages)
    elif CLIENT_TYPE == "groq":
        response = client.chat.completions.create(
            model='openai/gpt-oss-20b',
            messages=messages,
            stream=False,
            tools=tools,
            tool_choice="auto",
            max_completion_tokens=4096
        )

    process_query = True
    while process_query:
        if CLIENT_TYPE == "anthropic":
            assistant_content = []
            for content in response.content:
                if content.type == 'text':
                    
                    print(content.text)
                    assistant_content.append(content)
                    
                    if len(response.content) == 1:
                        process_query = False
                
                elif content.type == 'tool_use':
                    
                    assistant_content.append(content)
                    messages.append({'role': 'assistant', 'content': assistant_content})
                    
                    tool_id = content.id
                    tool_args = content.input
                    tool_name = content.name
                    print(f"Calling tool {tool_name} with args {tool_args}")
                    
                    result = execute_tool(tool_name, tool_args)
                    messages.append({"role": "user", 
                                      "content": [
                                          {
                                              "type": "tool_result",
                                              "tool_use_id": tool_id,
                                              "content": result
                                          }
                                      ]
                                    })
                        
                    response = client.messages.create(max_tokens = 2024,
                                      model = 'claude-3-7-sonnet-20250219', 
                                      tools = tools,
                                      messages = messages) 
                    
                    if len(response.content) == 1 and response.content[0].type == "text":
                        print(response.content[0].text)
                        process_query = False
                        
        elif CLIENT_TYPE == "groq":
            response_message = response.choices[0].message
            tool_calls = response_message.tool_calls
            if tool_calls:
                # Add the LLM's response to the conversation
                messages.append(response_message)
        
                # Process each tool call
                for tool_call in tool_calls:
                    tool_name = tool_call.function.name
                    tool_args = json.loads(tool_call.function.arguments)
                    print(f"Calling tool {tool_name} with args {tool_args}")
                    # Call the tool and get the response
                    result = execute_tool(tool_name, tool_args)
                    print("done function call")
                    # Add the tool response to the conversation
                    messages.append(
                        {
                            "tool_call_id": tool_call.id, 
                            "role": "tool", # Indicates this message is from tool use
                            "name": tool_name,
                            "content": result,
                        }
                    )
                # Make a second API call with the updated conversation
                response = client.chat.completions.create(
                    model='openai/gpt-oss-20b',
                    messages=messages
                )
                # Print the final response
                print(response.choices[0].message.content)
                process_query = False
            else:
                print(response.choices[0].message.content)
                process_query = False

### Chat Loop

In [48]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

Feel free to interact with the chatbot. Here's an example query: 

- Search for 2 papers on "LLM interpretability"

To access the `papers` folder: 1) click on the `File` option on the top menu of the notebook and 2) click on `Open` and then 3) click on `L3`.

In [49]:
chat_loop()

Type your queries or 'quit' to exit.



Query:  search for papers related with use of unsupervise leanrning applied to satellite data


ChatCompletion(id='chatcmpl-4d94740d-4efe-4d50-a4de-db3579753886', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning='We need to call function search_papers with topic "unsupervised learning applied to satellite data". Use default max_results 5.', tool_calls=[ChatCompletionMessageToolCall(id='fc_2514cbec-546f-4507-995d-808fea4b3f06', function=Function(arguments='{"max_results":5,"topic":"unsupervised learning applied to satellite data"}', name='search_papers'), type='function')]))], created=1759606160, model='openai/gpt-oss-20b', object='chat.completion', system_fingerprint='fp_77f8660d1d', usage=CompletionUsage(completion_tokens=64, prompt_tokens=209, total_tokens=273, completion_time=0.064492136, prompt_time=0.010216181, queue_time=0.195393011, total_time=0.074708317), usage_breakdown=None, x_groq={'id': 'req_01k6rb6ehyevebsttehyts7bqt'}, service_tier='on


Query:  what about the 1509.06095 paper founded?


ChatCompletion(id='chatcmpl-305635d0-5042-4afc-90f5-0a1876cf8bd1', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning='The user: "what about the 1509.06095 paper founded?" Likely they want information about arXiv paper 1509.06095. We need to use extract_info function to get info. Let\'s do that.', tool_calls=[ChatCompletionMessageToolCall(id='fc_f04850dc-cf2f-4891-bf39-afa25355adcc', function=Function(arguments='{"paper_id":"1509.06095"}', name='extract_info'), type='function')]))], created=1759606307, model='openai/gpt-oss-20b', object='chat.completion', system_fingerprint='fp_e99e93f2ac', usage=CompletionUsage(completion_tokens=77, prompt_tokens=204, total_tokens=281, completion_time=0.081540481, prompt_time=0.010578582, queue_time=0.196534669, total_time=0.092119063), usage_breakdown=None, x_groq={'id': 'req_01k6rbay8qf93amreg3f1n6kwd'}, service_tier='on_


Query:  what about the 1509.06095v1 paper founded?


ChatCompletion(id='chatcmpl-726fa44d-cb0d-43ff-a434-42b7dd8c8be4', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning='The user asks: "what about the 1509.06095v1 paper founded?" They likely refer to arXiv paper 1509.06095v1. We should look up details. Use search_papers or extract_info? The paper ID is given; we can use extract_info to get info about that specific paper.\n\nWe\'ll call functions.extract_info with paper_id "1509.06095v1".', tool_calls=[ChatCompletionMessageToolCall(id='fc_4ec0d608-2425-4f67-a0f4-06867cde6875', function=Function(arguments='{"paper_id":"1509.06095v1"}', name='extract_info'), type='function')]))], created=1759606431, model='openai/gpt-oss-20b', object='chat.completion', system_fingerprint='fp_3d587a02fb', usage=CompletionUsage(completion_tokens=117, prompt_tokens=206, total_tokens=323, completion_time=0.115452859, prompt_time=0.


Query:  quit


<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b> To Access the <code>requirements.txt</code> file or the <code>papers</code> folder: </b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em> and finally 3) click on <em>"L3"</em>.
</div>

In the next lessons, you will take out the tool definitions to wrap them in an MCP server. Then you will create an MCP client inside the chatbot to make the chatbot MCP compatible.  

## Resources

[Guide on how to implement tool use](https://docs.anthropic.com/en/docs/build-with-claude/tool-use/overview#how-to-implement-tool-use)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">


<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>